In [1]:
# torch imports
import torch
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    pred_threshold: 0.5 

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: 1 
    pin_memory: True 

In [ ]:
%%yaml calibration_cfg 

calibrator:
    _name: None 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

In [ ]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0 
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-area_estimation_error'
    # checkpoint: 'min-val-ece_loss'

## Gather Inference Options.

In [ ]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "ISLES_Eval"

base_model: 
    - "/storage/vbutoi/scratch/ESE/training/09_05_24_ISLES_MEGA"
    - "/storage/vbutoi/scratch/ESE/calibration/09_05_24_ISLES_LTS"
    - "/storage/vbutoi/scratch/ESE/calibration/09_05_24_ISLES_TS"

inference_data:
    _class: "ese.datasets.ISLES"
    split: ("cal", "val")

# We want to test if we can save preds.
log:
    save_preds: True 

dataloader:
    batch_size: 8

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [ ]:
len(inf_cfgs)

## Running Jobs

In [ ]:
from ese.analysis.run_inference import get_cal_stats

In [9]:
# from ese.experiment import run_ese_exp

# ###### Run individual jobs
# run_ese_exp(
#     config=inf_cfgs[-1], 
#     job_func=get_cal_stats,
#     run_name='debug',
#     # show_examples=True,
#     show_examples=True,
#     gpu='0',
#     # gpu='4',
# ) 

In [ ]:
from ese.experiment import submit_ese_exps 

#### Run Batch Jobs
submit_ese_exps(
    group="inference",
    base_cfg=updated_base_cfg,
    exp_cfg=experiment_cfg,
    config_list=inf_cfgs,
    job_func=get_cal_stats,
    available_gpus=['0', '1', '2', '3']
    # available_gpus=['4', '5', '6', '7']
)